#### Usamos el xgboost con los hiperparametros extraidos de 7.0 y agregamos la feature de si es lujosa o no.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt
import utils.encodeador as my_encoder
import utils.featurizerV2 as ft_v2

import re

from math import sqrt

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


In [3]:
df_test = pd.read_csv("data/test.csv")

In [4]:
train = ft_v2.featurizerv2(train,"train")
test = ft_v2.featurizerv2(test,"test")

In [5]:
train.shape

(240000, 56)

In [6]:
test.shape

(60000, 55)

In [7]:
drop_cols = ['id','titulo','descripcion','direccion','fecha']

In [8]:
X = train.drop(['precio'] + drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)
y = train['precio']

In [9]:
X,test = my_encoder.oneHotEncoder(X,test)

In [10]:
X.shape

(240000, 632)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
hyperOpt = {
    'objective' :'reg:squarederror', 
    'colsample_bytree' : 0.5, 
    'learning_rate' : 0.1,
    'max_depth' : 15,
    'alpha' : 10,
    'n_estimators' : 300,
    'eval_metric' : 'mae',
    'subsample' : 1,
    'min_child_weight' : 25,
    'gamma' : 5
}

In [26]:
xgb = XGBRegressor(**hyperOpt)

In [27]:
%%time
xgb.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 19min 22s, sys: 2.06 s, total: 19min 24s
Wall time: 19min 23s


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, eval_metric='mae',
             gamma=5, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=15, min_child_weight=25, missing=None,
             n_estimators=300, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
             subsample=1, verbosity=1)

In [28]:
pred = xgb.predict(X_test)

In [29]:
mean_absolute_error(y_test,pred)

529910.7930725098

In [32]:
predicion = xgb.predict(test)

In [33]:
resultado = pd.DataFrame(predicion, index=df_test.id, columns=['target'])
display(resultado.head())
resultado.to_csv("data/workshop-xgboost-v7_1.csv", header=True) 

,target
id,
4941,5738447.000
51775,1086590.625
115253,2112852.250
299321,1416033.625
173570,616590.250
